<a href="https://colab.research.google.com/github/Hongbi-Kim/Writing_lyrics/blob/master/GPT3_%EA%B2%B0%EA%B3%BC_%EB%8F%8C%EB%A0%A4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

- gpt3_1 : tokenizer 250, 300으로 학습
- gpt3_2 : tokenizer 350으로 학습
- gpt3_5000sample : 5000개 데이터, tokenizer 350으로 학습

In [ ]:
# 구글 드라이브에서 모델, 토크나이저 불러오기
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random
import re

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

# gpt3-kor-small_based_on_gpt2
from transformers import BertTokenizerFast, GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/5조_파이널PJT/코드/model_result/gpt3_2')
tokenizer_gpt3 = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/5조_파이널PJT/코드/model_result/gpt3_2')
input_ids = tokenizer_gpt3.encode("text to tokenize")[1:]  # remove cls token

In [ ]:
device = torch.device("cuda")
model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model = model.to(device)

In [ ]:
model.eval()
def lyric_generator(gen, lyric):
  prompt = f"<|startoftext|> <{gen}> {lyric}"
  generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
  generated = generated.to(device)

  print(generated)

  sample_outputs = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 350,
                                  #max_length = len(lyric) + 350,
                                  top_p=0.95, 
                                  num_return_sequences=1,
                                  repetition_penalty=1.1
                                  )

  for i, sample_output in enumerate(sample_outputs):
    result = "{}: {}\n\n".format(i, tokenizer_gpt3.decode(sample_output, skip_special_tokens=True))
    result = re.sub(r"<br>", "\n", result)
    #result = result[len(lyric)+1:]
    print(result)
  return

In [ ]:
lyric_generator('댄스', '매일 강아지랑 산책해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 14759, 14971,  8202, 16780,  8010,     3]],
       device='cuda:0')
0: 매일 강아지랑 산책해 
 oh oh oh ah ah ah ah ah 
 are you hear me guys 
 너에게 물었어 왜 이러냐고 
 what you think about 
 what you gotta do 
 왜 자꾸 나몰래 널 훔쳐가 
 난 알아 난 다 알아 난 다 알아 
 넌 내 눈빛에 담긴 표정 말투 말하는 거까지 
 알고싶어 미칠 것 같아 
 나 혼자 고민하지 마 no way 
 나 혼자 생각하지 마 no way 
 내 맘이 들릴지도 몰라 woo 
 나 혼자 흔들리면 어떡해 나 몰래 
 woo woo woo woo woo 
 woo woo woo woo woo 
 woo woo woo woo woo woo 
 날 안아줘 내게 말해줘 
 너도 나와 같다고 oh oh oh 
 매일 고양이랑 산책해줘 
 넌 가끔 멍 때리지 
 너무 귀여워서 훔쳐봐줘 
 장난치는 모습에 
 날 미치게 만들어 또 나도 몰래 
 자꾸 네게 전화하게 돼 
 내 맘 받아줄까 
 네 맘을 받아줄까 
 사실 난 겁이 났어 
 내가 먼저 다가갈까 봐 겁이 났어 
 난 알아 난 다 알아 난 다 알아 
 넌 너무 매력 있어 baby you and i 
 나 혼자 고민하지 마 no way 
 나 혼자 생각하지 마 no way 
 내 맘이 들릴지도 몰라 woo 
 나 혼자 흔들리면 어떡해 나 몰래 
 woo woo woo woo woo 
 woo woo woo woo woo woo 
 날 안아줘 내게 말해줘 
 너도 나와 같다고 oh oh o




In [ ]:
lyric_generator('발라드', '매일 강아지랑 산책해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003, 14759, 14971,  8202, 16780,  8010,     3]],
       device='cuda:0')
0: 매일 강아지랑 산책해 
 용기 내 전불 걸 수 있는 
 용길 줘요 함께해요 
 이 넓은 곳에서 우리 둘의 멋진 미래를 봐요 
 함께 달려갈래요 서로 손을 잡고서 
 우리 모두 함께 달려갈래요 우리 모두 
 별이 뜰 때까지 lets go 함께해요 
 별이 뜰 때까지 
 i love you i need you baby 
 별이 뜰 때까지 lets go 함께해요 
 너만의 빛나는 별을 찾아갈래 
 널 위한 별자리를 만들래 
 난 너만의 반짝이는 별이 될래 
 너만의 빛으로 밝게 빛나줘 
 너만의 환한 빛으로 
 밝게 빛나줘 
 너만의 빛나는 별을 찾을래 
 널 위한 별자리를 만들래 
 넌 날 깨워내는 저 빛의 마법사 
 저 높은 곳까지 올라 갈 것 같아 
 내 손에 잡힐 듯 말 듯이 잡히지가 않아 
 너의 주위를 공전하는 저 달이 
 이 어두운 밤하늘에 뜨는 별처럼 
 내가 지켜줄게요 
 같이 달려 갈래요 서로의 손 잡고서 
 너와 나 이렇게 둘이 저 빛나는 하늘을 향해가요 
 이 넓은 곳에서 우리 둘의 멋진 미래를 봐요 
 함께 달려갈래요 서로 손을 잡고서 
 우리 모두 함께 달려갈래요 우리 모두 
 별이 뜰 때까지 lets go 함께해요 
 별이 뜰 때까지 
 i love you i need you baby 
 별이 뜰 때까지 lets go 함께해요 
 함께 날아갈래요 
 ah ah ah ah ah ah ah 
 ah ah ah ah ah ah ah 
 같은 시간에 같은 시간에 함께라면 
 lets get a dream come true 
 함께 즐겨볼래요 hey 
 함께 날아갈래요 
 ah a




In [ ]:
len('주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 뛰어다녀요 <br> ')

36

In [ ]:
lyric_generator('댄스', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 뛰어다녀요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 16357, 22481,  8055,     3]], device='cuda:0')
0: 주말마다 강아지랑 산책해요 
 강아지는 너무 귀엽고 뛰어다녀요 
 바다로 갈래 어디로 갈지 몰라서 
 그 동안 못한 사랑 다시 찾았죠 
 이제는 더 이상 기다리기 지겨워지면 
 늦기전에 떠나요 멀리 떠나요 
 푸른 바다로 신나는 여름 속으로 
 푸른 바다로 신나는 여름 속으로 
 언제나 함께 있어줘요 
 파란 하늘 아래 모두 함께 떠나요 
 lets go summer 
 햇살이 부서지는 바닷가에 
 우리함께 달려가요 
 시원한 파도가 말해주는 시원한 여름 
 사랑하는 우리 영원히 사랑해 
 별이 쏟아지는 저 바다 위로 
 우리 함께 달려가요 
 저 푸른 바다로 가는 저 바다를 향해 
 손을 잡고 신나는 음악에 맞춰 
 행복을 느끼는 이 순간 
 저 바다가 보여 멋진 하늘이 보여 
 언제나 사랑을 비춰줘 빛나는 바다를 
 하얀 모래위에 예쁘게 적어 놓은 
 사랑해란 편지와 커다란 선물 
 푸른 바다로 신나는 여름 속으로 
 푸른 바다로 신나는 여름 속으로 
 언제나 함께 있어줘요 
 파란 하늘 아래 모두 함께 떠나요 
 lets go summer 
 햇살이 부서지는 바닷가에 
 우리함께 달려가요 
 시원한 파도가 말해주는 시원한 여름 
 사랑하는 우리 영원히 사랑해 
 별이 쏟아지는 저 바다 위에 
 우리함께 달려가요 
 저 푸른 바다로 가는 저 바다를 향해 
 손을 잡고 신나는 음악에 맞춰 
 행복을 느끼는 이 순간 
 저 바다가 보여 멋진 하늘이 보여 
 언제나 사랑을 비춰줘 빛나는 바다를 
 맑은바다를 바라보며 
 너와 난 바다를 가르고 싶었어 
 너의 마음을 내게줘 tell me baby 
 oh my lover 언제나 함께해 
 blue lover 언제나 함께있어 
 oh my love

In [ ]:
lyric_generator('댄스', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 <br> 폴짝 뛰어다녀요<br> 나는 행복해요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 42005,  7496,  8437, 16357, 22481,  8055, 42005, 14264,
         38380,  8055,     3]], device='cuda:0')
0: 주말마다 강아지랑 산책해요 
 강아지는 너무 귀엽고 
 폴짝 뛰어다녀요 
 나는 행복해요 
 내 손을 잡아 줄 사람 필요해 
 외롭게 하던 그 모든 것들을 
 나에 곁으로 와 바꿔준 이 사람 
 정말 고마워요 이 사랑을 믿어요 
 내 마음속 가득찬 그대를 향해 달려갈게요 
 모두 다 같이 떠나보자 하늘위로 
 우리 둘이 함께 하늘을 날아 가자 우리 둘이 함께 
 푸른 초원위에 그림 같은 집을 지어 살아봐요 
 가끔 눈이라도 오면 같이 걸어보고 
 멋진 밤에는 누구와 단 둘이 별을 보며 
 즐거운 상상을 해요 
 우리 모두 모여 하늘을 날아요 
 하늘 위로 
 파란하늘 아래서 자유롭게 살아봐요 
 항상 같은 하늘 아래서 함께해요 
 
 가끔 눈이라도 오면 같이 걸어보고 
 멋진 밤에는 누구와 단 둘이 별을 보며 
 즐거운 상상을 해요 
 우리 모두 모아 하늘을 날아요 
 하늘 위로 
 파란하늘 아래서 자유롭게 살아봐요 
 항상 같은 하늘 아래서 함께해요




In [ ]:
lyric_generator('발라드', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 <br> 폴짝 뛰어다녀요<br> 나는 행복해요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 42005,  7496,  8437, 16357, 22481,  8055, 42005, 14264,
         38380,  8055,     3]], device='cuda:0')
0: 주말마다 강아지랑 산책해요 
 강아지는 너무 귀엽고 
 폴짝 뛰어다녀요 
 나는 행복해요 
 어떡하죠 
 눈물이나요 너무 미안해져요 
 내 맘이 그댈 향한 것만 같아 
 너무 고마워요 사랑하게 됐나봐 
 love you love you love you 
 안아 주고 싶어요 
 시간이 지나도 
 영원토록 헤어지지 말아요 
 love you love you love you 
 그대만을 원해요 그대의 사랑 
 언제까지나 영원히 
 love you love you love you 
 기다려요 우리 함께 할거죠 
 눈물나는 날에도 
 나 항상 그대 곁에 있고 싶어 
 사랑해 사랑해 
 어떡하죠 눈물이나요 
 너무 미안해져요 
 내 맘이 그댈 향한 것만 같아 
 너무 고마워요 사랑하게 됐나봐 
 love you love you love you 
 안아 주고 싶어요 
 시간이 지나도 
 영원토록 헤어지지 말아요 
 love you love you love you 
 그대만을 원해요 그대의 사랑 
 언제까지나 영원히 
 love you love you love you 
 기다려요 우리 함께 할거죠 
 눈물나는 날에도 
 나 항상 그대 곁에 있고 싶어 
 사랑해 사랑해 
 언제까지나 영원히 
 love you love you love you 
 기다려요 우리 함께 할거죠 
 눈물나는 날에도 
 나 항상 그대 곁에 있고 싶어




In [ ]:
# 생성된 가사가 거의 영어인 경우
lyric_generator('댄스', '토요일 멘토링은 즐거워')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 26576,     3]], device='cuda:0')
0: 토요일 멘토링은 즐거워 
 오 그대와 함께라면 날아갈 수 있어 펼쳐진 넓은 저 우주를 향해서 
 oh love me tell me why dont you know how come on baby 
 god is your favorite girl so 
 youre just mine baby right now 
 everybody come on come on lets dance 
 right now right now 
 its my shooting star 
 my shooting star now everybody head like headlike headlike now 
 yeah 
 you will never let me fall in 
 until the end of time we holdin together with no hands 
 were back again bring this back 
 you will never let me fall in 
 until the end of time we holdin together with no hands 
 were back again bring this back 
 yo here we go oh man im a party with you boogie to the floor 
 uknow i should feel it hot tarot bust your body rock 
 hit the mic pound it up pump it up 
 we gonna make it now lets dance 
 right now right now 
 its my shooting star 
 my shooting star now everybody head like he




In [ ]:
# 훅만 반복되는 경우
lyric_generator('발라드', 'I love you')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003,  2049, 29808, 18641,     3]], device='cuda:0')
0: i love you 
 
 oh ah oh ah oh ah 
 
 wanna be yourself in the battle crazy like space around yeah yeah 
 
 파도 치는 바다를 헤엄쳐 
 그 속에 함께해 uh uh 
 파도치는 바다 위를 떠다닌 저 별들 중에 하나 
 
 i just hold my hand 
 
 i just dont know 
 
 i just see it 
 
 i just love me to think about you 
 
 we are all time getting party time 
 
 우리의 기쁨을 찾아주는 이 밤하늘에 달빛 
 
 i love you baby 
 
 oh ah oh ah oh ah ah 
 
 yeah yeah yeah yeah




In [ ]:
# 입력한 글은 안나오게 하기
lyric_result = lyric_generator('댄스', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 뛰어다녀요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 16357, 22481,  8055,     3]], device='cuda:0')
 
 이제는 때가 됐죠 
 i love you love you love you baby 
 love you love you love you baby 
 love you love you love you baby 
 내 곁에서 더 높이 날아갈래요 
 그대 이제 그만 좀 뛰놀까요 
 baby baby baby 내 맘에 들어와요 
 baby baby baby 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 날 사랑해줘요 
 언제나 함께 해요 
 i want your love 
 i need your love 
 i need your love 
 i need your love 
 내 곁에선 더 높이 날아갈래요 
 그대 이제 그만 좀 뛰놀까요 
 baby baby baby 내 맘에 들어와요 
 baby baby baby 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 날 사랑해줘요 
 언제나 함께 해요 
 가끔은 혼자도 괜찮아 둘이어도 괜찮아 
 난 괜찮으니까 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 날 사랑해줘요 
 언제나 함께 해요 
 baby baby baby 내게 와요 
 baby baby baby 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 내 사랑 받아줄래요 
 내 눈 속에 너를 담아 보낼게 
 love forev




In [ ]:
lyric_generator('댄스', '토요일 멘토링은 즐거워')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 26576,     3]], device='cuda:0')
0: 토요일 멘토링은 즐거워 
 조금씩 너를 향해 가고 있어 이젠 모든 것을 뒤로한 채 널 향해 달려가고 있어 
 어떤 어려운 것도 다 상관없어 그저 사랑하는 너의 손길만 
 나를 위해 준비된 오직 단 한사람만을 위한 시간이란걸 느끼며 
 자 시작해 볼까 내 자신을 돌아봐봐 너의 모든 사랑이 기다리는 곳에 다가서 봐 오 오 
 
 우리 서로 멀리 있어도 좋아 떠나자 지금 자유로워 이 기분을 느껴봐 
 
 이끌리는 대로 날개를 펴 세상을 날아가도 괜찮아 원하는 곳으로 
 꿈을 찾아 떠나보는 거야 baby one two 
 
 시원한 바람결에 두 팔을 벌리고




In [ ]:
lyric_generator('댄스', '토요일 멘토링은 신나')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 29863,     3]], device='cuda:0')
0: 토요일 멘토링은 신나 
 
 
 
 hey hey hey lets go 
 
 파란 하늘 위엔 멋진 별이 떠 있어요 
 
 
 
 우린 높이 날아갈 수 있어 우릴 따라와 줘 
 
 
 lets start this party 
 
 우리 모두 신나게 yeah we come on 
 
 이제 시작인데 lets bring it right now 
 
 소리쳐줘 크게 외쳐봐 
 
 
 
 푸른하늘 위를 달려보자 시원한 바람 불어와 
 
 
 hey hey hey lets go 
 
 하얀 파도위에 떠오른 태양이 눈부셔 
 
 
 
 저기 멀리 빛나는 저 별을 바라봐 
 
 
 
 푸른 바다속에 뛰어들어 저 깊은 밤을 날아봐 
 
 
 
 
 lets start this party 
 
 우리 모두 신나게 yeah we come on 
 
 이제 시작인데 lets bring it right now 
 
 소리쳐줘 크게 외쳐봐 
 
 
 
 푸른하늘 위를 달려보자 시원한 바람 불어와 
 
 
 
 hey hey hey lets go 
 
 하얀 파도위에 떠오른 태양이 눈부셔 
 
 
 
 저기 멀리 빛나는 저 별을 바라봐 
 
 
 
 푸른 바다속에 뛰어들어 저 깊은 밤을 날아봐 
 
 
 
 소리쳐줘 크게 외쳐봐 
 
 
 푸른 하늘 위를 달려보자 시원한 바람 불어와 
 
 
 
 하나 둘 셋 lets go 
 
 lets start this party 
 
 우리 모두 신나게 떠나볼래 
 
 
 
 푸른 바다 위로 띄워줄래 
 
 
 
 our summer fever 
 
 summer love together




In [ ]:
lyric_generator('댄스', '토요일 멘토링은 신나')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 29863,     3]], device='cuda:0')
0: 토요일 멘토링은 신나 
 오늘 밤엔 그대와 나만의 battle group star 
 그대와 단둘이 떠나요 둘이서 우리 단둘이 
 푸른 바다 위에 fishbacka bikini 
 이 밤이 새도록 너와 함께 
 뜨거운 태양아래 fishbacka bikini 
 내 맘을 모두 줄게 yeah 
 i wanna love you boy ooh ooh ooh 
 i wanna love you boy ooh ooh ooh 
 you love me love me boy 
 i love you boy love me boy 
 heartbeat heartbeat boy 
 heartbeat heartbeat boy 
 나를 한번 봐줘요 heartbeat heartbeat 
 heartbeat heartbeat boy 
 내 심장은 두근두근 뛰죠 난 매일이 생일인데 
 오늘만은 그대와 나뿐인 걸요 oh my lover 
 그대는 나의 shinee world 
 파란하늘 아래 sunshine 
 태양은 우릴 향해 비추고 
 달빛은 우릴 향해 비춰 
 우리 둘만 있는 그곳으로 oh my lover 
 우리의 별이 빛나는 이유는 
 우리만의 love story 
 이 밤이 새도록 너와 함께 
 뜨거운 태양아래 fishbacka bikini 
 이 밤이 새도록 너와 함께 
 뜨거운 태양아래 fishbacka bikini 
 내 맘을 모두 줄게 yeah 
 i wanna love you boy ooh ooh ooh 
 i


